# SentenceTransformer(bert 기반 모델) 사용하여 chat bot

질문을 model의 encoder를 이용하여 encoding -> encoding output으로 유사도 높은 것을 찾는다.
ChatBotData 에서 가장 유사도 높은 답변을 선택해준다.


In [1]:
pip install sentence_transformers


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer


/Users/kwangsiklee/miniforge3/envs/hf_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head(50)


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [3]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')


In [4]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)


In [5]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))


In [6]:
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

In [7]:
return_answer('너는 누구니?')


'저는 위로봇입니다.'

In [8]:
return_answer('화가납니다')


'화를 참는 연습을 해보세요.'

In [9]:
return_answer('배가 고프다')


'얼른 맛난 음식 드세요.'

In [10]:
return_answer('재미있는 얘기 해줘')

'부러워요. 좋은 관계를 유지하세요.'